In [ ]:
import csv
import datetime
import pickle
import joblib
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif

from xgboost import XGBClassifier

import optuna


In [ ]:
df = pd.read_pickle('C:\\diplomka_work\\python_code\\data_pickles\\train_cely.pkl')

In [ ]:




# Pridanie stĺpca `next_product_id`
df['next_product_name'] = df.groupby('doklad_id_int')['produkt_nazev'].shift(-1)

# Označenie, či je riadok posledný v objednávke
df['is_last'] = df['next_product_name'].isna()

# Odstránenie riadkov, kde je `is_last` True
df = df[~df['is_last']].copy()



# Odstránenie tried, ktoré majú menej ako 2 vzorky
class_counts = df['next_product_name'].value_counts()
valid_classes = class_counts[class_counts > 1].index
df = df[df['next_product_name'].isin(valid_classes)].copy()


In [ ]:
# Premenná 'column_names_dumm' obsahuje zoznam názvov stĺpcov, ktoré chceme transformovať na dummy premenné.
column_names_dumm = ['produkt_nazev']

# Funkcia pd.get_dummies() prevedie kategórie v stĺpci 'produkt_nazev' na samostatné binárne stĺpce (0 alebo 1).
# Výsledkom je rozšírený dataframe, kde každý jedinečný názov produktu bude reprezentovaný vlastným stĺpcom.
df = pd.get_dummies(df, columns = column_names_dumm)


In [ ]:
# Automatický výber všetkých čŕt okrem identifikátorov a cieľových premenných
exclude_columns = ['doklad_id', 'produkt_id', 'item_sequence', 'poradi', 'next_product_name', 'next_product_id_encoded', 'is_last','otevreni_datum_cas','produkt_nazev', 'produkt_id_int', 'doklad_id_int']
features = [col for col in df.columns if col not in exclude_columns]

X = df[features]
y = df['next_product_name']

In [ ]:


# Oprava názvov stĺpcov (odstránenie špeciálnych znakov)
X.columns = [re.sub(r'[^A-Za-z0-9_]', '_', col) for col in X.columns]

In [ ]:
# Skontroluj dátové typy
print("Dátové typy v DataFrame:")
print(df.dtypes)

# Nájdeme stĺpce, ktoré nie sú int, float alebo bool
problematic_columns = df.select_dtypes(exclude=['int64', 'float64', 'bool']).columns.tolist()

# Výpis problematických stĺpcov
if problematic_columns:
    print("\n⚠️  Stĺpce s nesprávnym typom (môžu spôsobiť chybu v modelovaní):")
    print(problematic_columns)
else:
    print("\n✅  Všetky stĺpce sú v správnom formáte pre modelovanie.")

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# --- Rozdelenie dát ---
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# --- Label Encoding cieľa ---
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_val_enc = encoder.transform(y_val)

# --- Optimalizačná funkcia ---
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'n_jobs': -1,
        'tree_method': 'hist',
        'random_state': 42
    }

    model = XGBClassifier(**params)

    model.fit(
        X_train, y_train_enc,
        eval_set=[(X_val, y_val_enc)],
        early_stopping_rounds=50,
        verbose=False
    )

    preds = model.predict(X_val)
    acc = accuracy_score(y_val_enc, preds)
    return acc

# --- Optimalizácia + ukladanie výsledkov ---
study = optuna.create_study(direction='maximize')

try:
    study.optimize(objective, n_trials=20)
finally:
    # --- Uloženie všetkých trialov do TXT ---
    with open("optuna_trials.txt", "w") as f:
        for trial in study.trials:
            f.write(f"Trial #{trial.number}\n")
            f.write(f"  Value: {trial.value}\n")
            f.write("  Params:\n")
            for key, val in trial.params.items():
                f.write(f"    {key}: {val}\n")
            f.write("\n")

    # --- Uloženie najlepších parametrov do TXT ---
    with open("best_params.txt", "w") as f:
        f.write("Najlepšie parametre:\n")
        for key, val in study.best_trial.params.items():
            f.write(f"{key}: {val}\n")
        f.write(f"\nNajlepšia presnosť: {study.best_value:.4f}\n")

    print("Výsledky boli uložené do 'optuna_trials.txt' a 'best_params.txt'.")

# --- Výpis najlepší výsledok aj na konzolu ---
print("Najlepšie parametre:")
print(study.best_params)
print(f"Najlepšia presnosť: {study.best_value:.4f}")


In [ ]:

# --- Label encoding cieľa ---
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)


In [ ]:





# --- Najlepšie parametre z Optuny (Trial 9) ---
best_params = {
    'n_estimators': 1625,
    'max_depth': 12,
    'learning_rate': 0.007688491612625464,
    'subsample': 0.7642837248864608,
    'colsample_bytree': 0.7564773901258157,
    'gamma': 0.9237326806390628,
    'reg_alpha': 0.19162436915031944,
    'reg_lambda': 9.912174793053591,
    'min_child_weight': 6,
    'n_jobs': -1,
    'random_state': 42,
    'verbosity': 1,
    'tree_method': 'hist'
}

# --- Trénovanie modelu ---
final_model = XGBClassifier(**best_params)

final_model.set_params(early_stopping_rounds=80)





# --- Vyhodnotenie ---
final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"Final XGBoost Accuracy: {final_accuracy:.4f}")


In [ ]:
final_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)


In [ ]:
# --- Predikcia ---
y_pred_final = final_model.predict(X_test)


In [ ]:

# --- Vyhodnotenie ---
final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"Final XGBoost Accuracy: {final_accuracy:.4f}")